In [1]:
import os
if "py" not in os.listdir("."):
    os.chdir("..")
    
%config Completer.use_jedi = False

In [2]:
import microdf as mdf
import numpy as np
import openfisca_uk as o
import pandas as pd
from openfisca_uk import IndividualSim, PopulationSim
from openfisca_uk.reforms.modelling import reported_benefits
from py.calc_ubi import ubi_reform

REGIONS = [
    "NORTH_EAST",
    "NORTH_WEST",
    "YORKSHIRE",
    "EAST_MIDLANDS",
    "WEST_MIDLANDS",
    "EAST_OF_ENGLAND",
    "LONDON",
    "SOUTH_EAST",
    "SOUTH_WEST",
    "WALES",
    "SCOTLAND",
    "NORTHERN_IRELAND",
]

region_map = dict(zip(range(len(REGIONS)), REGIONS))

optimal_params = pd.read_csv("optimal_params.csv")  # Up a folder.


def reform(i):
    row = optimal_params.iloc[i].round()
    return ubi_reform(
        adult=row.adult,
        child=row.child,
        senior=row.senior,
        dis_base=row.dis_base,
        geo=row[REGIONS],
    )


reforms = [reform(i) for i in range(3)]

baseline_sim = PopulationSim(reported_benefits)
reform_sims = [PopulationSim(reported_benefits, reform) for reform in reforms]

REFORM_NAMES = ["1: Foundational", "2: Disability", "3: Disability + geo"]

BASELINE_PERSON_COLS = [
    "household_weight",
    "age",
    "region",
    "is_disabled_for_ubi",
]

# Extract these for baseline too.
REFORM_PERSON_COLS = ["household_net_income", "in_poverty_bhc", "in_deep_poverty_bhc"]

BASELINE_HH_COLS = [
    "household_weight",
    "poverty_gap_bhc",
    "poverty_gap_ahc",
    "people_in_household",
]

# Extract these for baseline too.
REFORM_HH_COLS = [
    "household_net_income",
    "equiv_household_net_income",
    "poverty_gap_bhc",
    "poverty_gap_ahc",
]

p_base = mdf.MicroDataFrame(
    baseline_sim.df(BASELINE_PERSON_COLS + REFORM_PERSON_COLS, map_to="person"),
    weights="household_weight",
)
p_base.rename(
    dict(zip(REFORM_PERSON_COLS, [i + "_base" for i in REFORM_PERSON_COLS])),
    axis=1,
    inplace=True,
)

hh_base = mdf.MicroDataFrame(
    baseline_sim.df(BASELINE_HH_COLS + REFORM_HH_COLS, map_to="household"),
    weights="household_weight",
)
hh_base.rename(
    dict(zip(REFORM_HH_COLS, [i + "_base" for i in REFORM_HH_COLS])),
    axis=1,
    inplace=True,
)
hh_base["person_weight"] = hh_base.household_weight * hh_base.people_in_household

# Change weight column to represent people for decile groups.
hh_base.set_weights(hh_base.person_weight)
hh_base["decile"] = np.ceil(hh_base.equiv_household_net_income_base.rank(pct=True) * 10)

# Change weight back to household weight for correct calculation of totals.
hh_base.set_weights(hh_base.household_weight)


def reform_p(i):
    p = reform_sims[i].df(REFORM_PERSON_COLS, map_to="person")
    p["reform"] = REFORM_NAMES[i]
    return mdf.concat([p_base, p], axis=1)


def reform_hh(i):
    hh = reform_sims[i].df(REFORM_HH_COLS, map_to="household")
    hh["reform"] = REFORM_NAMES[i]
    return mdf.concat([hh_base, hh], axis=1)

In [3]:
pd.Series(hh_base.household_weight).sum()

27828788.0

In [8]:
decile = (
        # Remove non-positive income, for whom income change is unhelpful.
        h[h.household_net_income_base > 0]
        .groupby(["reform", "decile"])[
            [
                "household_net_income_base",
                "household_net_income",
                "household_weight",
            ]
        ]
        .sum()
        .reset_index()
    )
decile["chg"] = (
    decile.household_net_income - decile.household_net_income_base
)
decile["chg_per_hh"] = decile.chg / decile.household_weight
decile["pc"] = decile.chg / decile.household_net_income_base

NameError: name 'h' is not defined

In [5]:
pd.DataFrame(hh_base).groupby("decile").person_weight.sum()

decile
1.0     6544922.0
2.0     6549727.0
3.0     6544810.0
4.0     6546580.0
5.0     6547964.0
6.0     6552148.0
7.0     6546026.0
8.0     6548129.0
9.0     6541254.0
10.0    6556995.0
Name: person_weight, dtype: float32

In [6]:
hh_base.equiv_household_net_income_base < (295 * 52)

0        False
1        False
2         True
3        False
4        False
         ...  
19164    False
19165    False
19166    False
19167    False
19168    False
Name: equiv_household_net_income_base, Length: 19169, dtype: bool

In [7]:
((hh_base.equiv_household_net_income_base < (295 * 52)) != hh_base.in_poverty_bhc).mean()

AttributeError: 'MicroDataFrame' object has no attribute 'in_poverty_bhc'

In [ ]:
(hh_base.equiv_household_net_income_base < hh_base.poverty_line_bhc) != hh_base.in_poverty_bhc

In [ ]:
o.tools.aggregates.poverty_rate(baseline_sim)